# email 

In [ ]:
19157521820@163.com

18285998868@163.com

3124568493@qq.com

kefu1820@gmail.com

# cryptomator

In [ ]:
%%bash
yay -Sy --needed --noconfirm cryptomator

# download link: https://cryptomator.org/downloads/#linux
# keepassdb 
# advise rub clean goodbye sketch terrain maximum recycle hi report theme viable length wonderful consent roughly remind overly slice available candidate sincere satisfy shrink yet elite elite secretary contract matching choir implement number through secondary ashamed pulse nonsense widen so ancestor pain chemistry ritual

# kepasskey
# strive stamp lab delighted silent potential interfere obesity issue eastern situated detail recommend municipal sweet casual chocolate pose integrate support tear notion normally current react nonsense sit pride fresh tender hall excited soil intimate cannot convince slowly sail least grab minister shy grief poverty


# nodejs npm

In [ ]:
!sudo pacman -Syu --needed --noconfirm nodejs npm

# add brtfs part to fstab

In [ ]:
%%bash
# echo "Adding /data partition"
# lsblk -f
# sudo mkdir -p /data && echo "UUID=180161ab-546d-4dfe-969e-e1cf9938195e /data btrfs defaults,compress=zstd,noatime 0 0" | sudo tee -a /etc/fstab

# systemctl daemon-reload
# sudo mount -a
# df -h | grep /data
# sudo chown $USER:$USER /data
# sudo chmod 755 /data


# btrfs创建系统快照教程

In [ ]:
%%bash
# 创建快照
# 检查分区的子卷
echo "检查根分区的子卷"
sudo btrfs subvolume list /
echo "检查 /home 分区的子卷"
sudo btrfs subvolume list /home
echo "检查 /data 分区的子卷"
sudo btrfs subvolume list /data

echo "为根分区、/home 分区和 /data 分区创建快照目录"
sudo mkdir -p /.snapshots
sudo mkdir -p /home/.snapshots
sudo mkdir -p /data/.snapshots


echo "为根分区创建只读快照"
TIMESTAMP=$(date +%Y%m%d_%H%M%S)
sudo btrfs subvolume snapshot -r / /.snapshots/roor_${TIMESTAMP}
echo "为 /home 分区创建只读快照"
sudo btrfs subvolume snapshot -r /home /home/.snapshots/home_${TIMESTAMP}
echo "为 /data 分区创建只读快照"
sudo btrfs subvolume snapshot -r /data /data/.snapshots/data_${TIMESTAMP}


In [ ]:
%%bash
echo "为 / 分区创建只读快照"
TIMESTAMP=$(date +%Y%m%d_%H%M%S)
sudo btrfs subvolume snapshot -r / /.snapshots/roor_${TIMESTAMP}
echo "为 /home 分区创建只读快照"
sudo btrfs subvolume snapshot -r /home /home/.snapshots/home_${TIMESTAMP}
echo "为 /data 分区创建只读快照"
sudo btrfs subvolume snapshot -r /data /data/.snapshots/data_${TIMESTAMP}

In [ ]:
%%bash
echo "=== 根分区快照 ==="
sudo btrfs subvolume list / | grep snapshots
echo "=== 家目录分区快照 ==="
sudo btrfs subvolume list /home | grep snapshots
echo "=== 数据分区快照 ==="
sudo btrfs subvolume list /data | grep snapshots

In [ ]:
%%bash
echo "=== 删除根目录所有快照 ==="
sudo btrfs subvolume delete /.snapshots/*
echo "=== 删除 home 目录所有快照 ==="
sudo btrfs subvolume delete /home/.snapshots/*
echo "=== 删除 data 目录所有快照 ==="
sudo btrfs subvolume delete /data/.snapshots/*

# 从快照恢复的完整流程

In [ ]:
%%bash
#   从 Live USB 启动并挂载所有分区
# 创建挂载点
sudo mkdir -p /mnt/{root,home,data}

# 挂载根分区
sudo mount /dev/nvme0n1p1 /mnt/root

# 挂载家目录分区
sudo mount /dev/nvme0n1p3 /mnt/home

# 挂载数据分区（可选）
sudo mount /dev/nvme0n1p5 /mnt/data

cd /mnt/root
# 删除损坏的子卷
sudo btrfs subvolume delete @
sudo btrfs subvolume delete @cache  
sudo btrfs subvolume delete @log
# 从快照恢复
sudo btrfs subvolume snapshot .snapshots/root_20241027_153000_ro @
sudo btrfs subvolume snapshot .snapshots/cache_20241027_153000_ro @cache
sudo btrfs subvolume snapshot .snapshots/log_20241027_153000_ro @log



# 恢复家目录分区
cd /mnt/home

# 先检查家目录的子卷名称
sudo btrfs subvolume list .

# 假设子卷名是 @home，删除并恢复
sudo btrfs subvolume delete @home
sudo btrfs subvolume snapshot .snapshots/home_20241027_153000_ro @home

# 恢复数据分区（可选）
cd /mnt/data

# 先检查数据目录的子卷名称
sudo btrfs subvolume list .

# 假设子卷名是 @data，删除并恢复
sudo btrfs subvolume delete @data
sudo btrfs subvolume snapshot .snapshots/data_20241027_153000_ro @data

# 卸载所有分区
cd /
sudo umount /mnt/{root,home,data}
sudo rmdir /mnt/{root,home,data}

# 重启系统
sudo reboot

# git 配置

In [ ]:
%%bash
git init
git remote add origin git@github.com:luckykefu/ManjaroUsingLog.git


In [ ]:
%%bash
git add .
git commit -m "auto run"
git push


# auto run git sync

In [ ]:
!sudo systemctl enable --now cronie
!systemctl status cronie

In [ ]:
%%bash
auto_sync_sh=$(pwd)/git_auto_sync.sh
echo $auto_sync_sh
cat > $auto_sync_sh << 'EOF'
#!/bin/bash
cd /data/Manjaro
git add .
git commit -m "auto run $(date '+%Y-%m-%d %H:%M:%S')"
git push
EOF
echo "add x quanxian"
chmod +x $auto_sync_sh

log_file=$(pwd)/log/git_sync.log
mkdir -p $(dirname $log_file)
# 删除旧的 git_auto_sync 任务，添加新任务
(crontab -l 2>/dev/null | grep -v "git_auto_sync.sh"; echo "*/30 * * * * $auto_sync_sh >> $log_file 2>&1") | crontab -
echo "=== 当前定时任务 ==="
crontab -l
# tail -f $log_file

# software

# docker docker-compose

In [ ]:
%%bash
sudo pacman -Sy --needed --noconfirm docker docker-compose
sudo groupadd docker
sudo usermod -aG docker $USER
# 配置 Docker 镜像加速器（中国用户） https://status.anye.xyz/
sudo mkdir -p /etc/docker
sudo tee /etc/docker/daemon.json <<-'EOF'
{
  "registry-mirrors": [
    "https://docker.1ms.run",
    "https://docker.1panel.live",
    "https://docker.m.ixdev.cn",
    "https://hub.rat.dev",
    "https://dockerproxy.net",
    "https://docker.hlmirror.com",
    "https://hub1.nat.tf",
    "https://hub3.nat.tf",
    "https://docker.m.daocloud.io",
    "https://docker.kejilion.pro",
    "https://hub.1panel.dev",
    "https://dockerproxy.cool",
    "https://proxy.vvvv.ee"
  ]
}
EOF
# 重启 Docker 服务
sudo systemctl daemon-reload
sudo systemctl restart docker
echo "Docker 配置完成，重启生效"


In [ ]:
%%bash
sudo tee /etc/docker/daemon.json <<-'EOF'
{
  "registry-mirrors": [
    "https://docker.1ms.run",
    "https://docker.1panel.live",
    "https://docker.m.ixdev.cn",
    "https://hub.rat.dev",
    "https://dockerproxy.net",
    "https://docker.hlmirror.com",
    "https://hub1.nat.tf",
    "https://hub3.nat.tf",
    "https://docker.m.daocloud.io",
    "https://docker.kejilion.pro",
    "https://hub.1panel.dev",
    "https://dockerproxy.cool",
    "https://proxy.vvvv.ee"
  ]
}
EOF
# 重启 Docker 服务
sudo systemctl daemon-reload
sudo systemctl restart docker

# ventoy

In [ ]:
!sudo pacman -Sy --needed --noconfirm ventoy

#  Davinci Resolve

In [ ]:
# download link: https://www.blackmagicdesign.com/products/davinciresolve/

# shadowsocks

In [ ]:
%%bash
## 服务器端manjaro版本
sudo pacman -Sy shadowsocks --noconfirm --needed
## 设置配置变量
SERVER="0.0.0.0"
PORT=8388
PASSWORD="vpntest"
METHOD="aes-256-gcm"
TIMEOUT=300

## 创建配置文件
sudo mkdir -p /etc/shadowsocks
sudo tee /etc/shadowsocks/config.json > /dev/null << EOF
{
    "server": "$SERVER",
    "server_port": $PORT,
    "password": "$PASSWORD",
    "method": "$METHOD",
    "timeout": $TIMEOUT,
    "fast_open": false,
    "mode": "tcp_and_udp"
}
EOF

## 验证配置
sudo cat /etc/shadowsocks/config.json
sudo systemctl start shadowsocks-server@config
sudo systemctl status shadowsocks-server@config

## 添加 iptables 规则开放8388端口
sudo iptables -A INPUT -p tcp --dport 8388 -j ACCEPT
sudo iptables -A INPUT -p udp --dport 8388 -j ACCEPT

## 保存 iptables 规则（根据系统选择合适的方法）
sudo iptables-save | sudo tee /etc/iptables/iptables.rules

## 检查规则是否添加成功
sudo iptables -L INPUT -n | grep 8388

## 成功


In [ ]:
%%bash
# 客户端
sudo pacman -Sy shadowsocks --noconfirm --needed
SERVER="207.148.77.242"
PASSWORD="vpntest"
LOCAL_PORT=1080

sudo tee /etc/shadowsocks/config.json > /dev/null << EOF
{
    "server": "$SERVER",
    "server_port": 8388,
    "local_address": "127.0.0.1",
    "local_port": "$LOCAL_PORT",
    "password": "$PASSWORD",
    "method": "aes-256-gcm",
    "timeout": 300
}
EOF
cat /etc/shadowsocks/config.json
sudo systemctl start shadowsocks@config 
sudo systemctl status shadowsocks@config   

curl --socks5 127.0.0.1:1080 ipinfo.io/ip
sudo systemctl stop shadowsocks@config  

In [ ]:
%%bash
# ssh: connect to host 207.148.77.242 port 22: Connection refused
ssh-keygen -R 207.148.77.242 

# ollama


In [ ]:
import os
from pathlib import Path
import subprocess

# 创建安装目录
install_dir = os.path.join(os.getcwd(), 'user')
ollama_dir = os.path.join(install_dir, '.ollama')
ollama_bin_dir = os.path.join(ollama_dir, 'bin')
ollama_models_dir = os.path.join(ollama_dir, 'models')

# 创建目录
Path(ollama_bin_dir).mkdir(parents=True, exist_ok=True)
Path(ollama_models_dir).mkdir(parents=True, exist_ok=True)

# 下载并安装 Ollama
# os.system(f'wget https://github.com/ollama/ollama/releases/latest/download/ollama-linux-amd64.tgz -O {install_dir}/ollama-linux-amd64.tgz')
# os.system(f'tar -xzf {install_dir}/ollama-linux-amd64.tgz -C {install_dir}/ollama/')
# os.system(f'rm -f {install_dir}/ollama-linux-amd64.tgz')

# 添加到 PATH
from src.add_path2shrc import PathManager
path_manager = PathManager()
path_manager.add_path(ollama_bin_dir)


In [ ]:
## AMD 显卡驱动 安装

In [ ]:
%%bash
## ## 安装 ollama ROCm lib
## wget https://github.com/ollama/ollama/releases/latest/download/ollama-linux-amd64-rocm.tgz -O ollama-linux-amd64-rocm.tgz

## 解压 ROCm 库到 Ollama 目录
# tar -xzf ollama-linux-amd64-rocm.tgz -C bin/ollama/



# aria2 webui
https://github.com/mayswind/AriaNg/releases

In [ ]:
import os
from pathlib import Path

def setup_aria2_config():
    """Setup aria2 configuration directory and files"""
    
    ## Create config directory
    config_dir = Path.home() / ".config" / "aria2"
    config_dir.mkdir(parents=True, exist_ok=True)
    print(f"📁 Created config directory: {config_dir}")
    
    ## Create session file
    session_file = config_dir / "aria2.session"
    session_file.touch()
    print(f"📄 Created session file: {session_file}")
    
    ## Create configuration file
    config_content = """## 基本设置
dir=${HOME}/Downloads
continue=true
max-connection-per-server=16
min-split-size=1M
split=16

## 进度保存
input-file=${HOME}/.config/aria2/aria2.session
save-session=${HOME}/.config/aria2/aria2.session
save-session-interval=60

## RPC 设置 (用于 WebUI)
enable-rpc=true
rpc-listen-all=true
rpc-listen-port=6800
rpc-secret=mysecret

## BT 设置
bt-enable-lpd=true
bt-max-peers=50
bt-request-peer-speed-limit=1M
bt-stop-timeout=0

## 性能优化
max-overall-download-limit=0
max-overall-upload-limit=1K
seed-ratio=1.0
seed-time=0

## 日志
log=${HOME}/.config/aria2/aria2.log
log-level=warn
"""
    
    config_file = config_dir / "aria2.conf"
    with open(config_file, 'w', encoding='utf-8') as f:
        f.write(config_content)
    print(f"⚙️  Created config file: {config_file}")
    
    print("✅ aria2 configuration completed")

## Run the function
setup_aria2_config()

In [ ]:
import os
from pathlib import Path
import subprocess

def setup_aria2_service():
    """Setup aria2 as a systemd user service"""
    
    ## Create systemd user directory
    service_dir = Path.home() / ".config" / "systemd" / "user"
    service_dir.mkdir(parents=True, exist_ok=True)
    print(f"📁 Created service directory: {service_dir}")
    
    ## Create service file content
    service_content = """[Unit]
Description=Aria2c Download Manager
After=network.target

[Service]
Type=forking
ExecStart=/usr/bin/aria2c --conf-path=%h/.config/aria2/aria2.conf -D
ExecStop=/usr/bin/killall aria2c
Restart=on-failure

[Install]
WantedBy=default.target
"""
    
    ## Write service file
    service_file = service_dir / "aria2.service"
    with open(service_file, 'w', encoding='utf-8') as f:
        f.write(service_content)
    print(f"⚙️  Created service file: {service_file}")
    
    ## Reload systemd and enable service
    try:
        subprocess.run(["systemctl", "--user", "daemon-reload"], check=True)
        print("✅ Systemd daemon reloaded")
        
        subprocess.run(["systemctl", "--user", "enable", "aria2"], check=True)
        print("✅ Service enabled")
        
        subprocess.run(["systemctl", "--user", "start", "aria2"], check=True)
        print("✅ Service started")
        
        print("🎉 aria2 system service installed successfully")
        
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to setup service: {e}")
        return False
    
    return True

## Run the function
setup_aria2_service()

In [ ]:
%%bash
## 不使用 jq，通过 grep 和 sed 提取版本号
LATEST_TAG=$(curl -s "https://api.github.com/repos/mayswind/AriaNg/releases/latest" | grep '"tag_name":' | sed -E 's/.*"([^"]+)".*/\1/')
echo "最新版本: $LATEST_TAG"

wget "https://github.com/mayswind/AriaNg/releases/download/${LATEST_TAG}/AriaNg-${LATEST_TAG}-AllInOne.zip"

echo "✅ 下载完成"

## unzip aria2 webui
unzip -o AriaNg-${LATEST_TAG}-AllInOne.zip -d ./bin/AriaNg

rm -f "AriaNg-${LATEST_TAG}-AllInOne.zip"

python3 -m http.server 8080 --directory ./bin/AriaNg


- 打开浏览器访问 http://localhost:8080/
- AriaNg 设置
- RPC
- Aria2 RPC 协议
- websocket
- 密钥

# yabridge

In [ ]:


```bash
sudo pacman -Sy --needed --noconfirm yabridge yabridgectl wine wine-gecko wine-mono  winetricks 

## Acon Digital
WINEPREFIX=/run/media/kf/WD/wine/vst/AconDigital winecfg
yabridgectl add /run/media/kf/WD/wine/vst/AconDigital/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## Antares
No success
WINEPREFIX=/run/media/kf/WD/wine/vst/AntaresAutoTune winetricks dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/AntaresAutoTune wine Antares\ Auto-Tune\ Pro\ v11.0.0.exe 
yabridgectl add /run/media/kf/WD/wine/vst/AntaresAutoTune/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/AntaresDeEsser wine 
yabridgectl add /run/media/kf/WD/wine/vst/AntaresDeEsser/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## Arturia
WINEPREFIX=/run/media/kf/WD/wine/vst/Arturia winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/Arturia wine 
yabridgectl add /run/media/kf/WD/wine/vst/Arturia/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## BABY Audio
WINEPREFIX=/run/media/kf/WD/wine/vst/BABYAudio winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/BABYAudio winecfg
yabridgectl add /run/media/kf/WD/wine/vst/BABYAudio/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

##BBE  
WINEPREFIX=/run/media/kf/WD/wine/vst/BBESonicSweet winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/BBESonicSweet wine s
yabridgectl add /run/media/kf/WD/wine/vst/BBESonicSweet/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/BBEStompBoard  winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/BBEStompBoard wine s
yabridgectl add /run/media/kf/WD/wine/vst/BBEStompBoard/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## Boz Digital Labs
WINEPREFIX=/run/media/kf/WD/wine/vst/BozDigitalLabs wine 
yabridgectl add /run/media/kf/WD/wine/vst/BozDigitalLabs/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## CableGuys
WINEPREFIX=/run/media/kf/WD/wine/vst/CableGuysShaperBox winetricks dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/CableGuysShaperBox wine 
yabridgectl add /run/media/kf/WD/wine/vst/CableGuysShaperBox/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/CableGuysKickStart winetricks dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/CableGuysKickStart wine 
yabridgectl add /run/media/kf/WD/wine/vst/CableGuysKickStart/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## CamelCrusher
WINEPREFIX=/run/media/kf/WD/wine/vst/CamelCrusher wine 
yabridgectl add /run/media/kf/WD/wine/vst/CamelCrusher/drive_c/Program\ Files/VSTPlugins/
yabridgectl sync --prune

## Celemony
WINEPREFIX=/run/media/kf/WD/wine/vst/Celemony winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/Celemony wine 
yabridgectl add /run/media/kf/WD/wine/vst/Celemony/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## cradle
WINEPREFIX=/run/media/kf/WD/wine/vst/Cradle winetricks dxvk 
WINEPREFIX=/run/media/kf/WD/wine/vst/Cradle wine 
yabridgectl add /run/media/kf/WD/wine/vst/Cradle/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## d16grouprepeater
WINEPREFIX=/run/media/kf/WD/wine/vst/D16GroupRepeater winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/D16GroupRepeater wine s
yabridgectl add /run/media/kf/WD/wine/vst/D16GroupRepeater/drive_c/Program\ Files//Steinberg/VSTPlugins/
yabridgectl sync --prune

##dadalife
no success

## deniseaudio
WINEPREFIX=/run/media/kf/WD/wine/vst/DeniseAudio winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/DeniseAudio wine 
yabridgectl add /run/media/kf/WD/wine/vst/DeniseAudio/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## exciteaudio
WINEPREFIX=/run/media/kf/WD/wine/vst/ExciteAudioHarmonic winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/ExciteAudioHarmonic wine s
yabridgectl add /run/media/kf/WD/wine/vst/ExciteAudioHarmonic/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune


## fabfilter
WINEPREFIX=/run/media/kf/WD/wine/vst/FabFilter winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/FabFilter wine s
yabridgectl add /run/media/kf/WD/wine/vst/FabFilter/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## goodhertz
WINEPREFIX=/run/media/kf/WD/wine/vst/Goodhertz winetricks mfc42 
WINEPREFIX=/run/media/kf/WD/wine/vst/Goodhertz wine 
yabridgectl add /run/media/kf/WD/wine/vst/Goodhertz/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune
WINEPREFIX=/run/media/kf/WD/wine/vst/Goodhertz wine cmd /c 
WINEPREFIX=/run/media/kf/WD/wine/vst/Goodhertz wine regedit 

## Harman Audio
WINEPREFIX=/run/media/kf/WD/wine/vst/LexiconLXP winetricks mfc42 
WINEPREFIX=/run/media/kf/WD/wine/vst/LexiconLXP wine 
yabridgectl add /run/media/kf/WD/wine/vst/LexiconLXP/drive_c/Program\ Files/Steinberg/VSTPlugins/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/LexiconPCM winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/LexiconPCM wine 
yabridgectl add /run/media/kf/WD/wine/vst/LexiconPCM/drive_c/Program\ Files/Steinberg/VSTPlugins/
yabridgectl sync --prune

## Heckmann Audio - u-he Diva v1.4.8
WINEPREFIX=/run/media/kf/WD/wine/vst/uheDiva wine 
yabridgectl add /run/media/kf/WD/wine/vst/uheDiva/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## ikmultimedia
WINEPREFIX=/run/media/kf/WD/wine/vst/IKMultimediaTracks6 winetricks mfc42 dxvk  
WINEPREFIX=/run/media/kf/WD/wine/vst/IKMultimediaTracks6 wine 
yabridgectl add /run/media/kf/WD/wine/vst/IKMultimediaTracks6/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## izotope
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeAurora winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeAurora wine 
yabridgectl add /run/media/kf/WD/wine/vst/IzotopeAurora/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeRX11 winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeRX11 wine 
yabridgectl add /run/media/kf/WD/wine/vst/IzotopeRX11/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune
##############################
## black screen
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeOzone11 winetricks mfc42 
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeOzone11 wine 
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeOnectar winetricks mfc42 
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeOnectar wine
##############################

## MinimalAudio
WINEPREFIX=/run/media/kf/WD/wine/vst/MinimalAudio winetricks  dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/MinimalAudio wine 
yabridgectl add /run/media/kf/WD/wine/vst/MinimalAudio/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## Native Instruments
WINEPREFIX=/run/media/kf/WD/wine/vst/NativeInstruments winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/NativeInstruments wine *.exe
yabridgectl add /run/media/kf/WD/wine/vst/NativeInstruments/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## NembriniAudio

WINEPREFIX=/run/media/kf/WD/wine/vst/NembriniAudio winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/NembriniAudio wine 
yabridgectl add /run/media/kf/WD/wine/vst/NembriniAudio/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## okesound
WINEPREFIX=/run/media/kf/WD/wine/vst/Okesound winecfg
yabridgectl add /run/media/kf/WD/wine/vst/Okesound/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## overloud 
WINEPREFIX=/run/media/kf/WD/wine/vst/Overloud winetricks dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/Overloud wine 
yabridgectl add /run/media/kf/WD/wine/vst/Overloud/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## pluginalliance
WINEPREFIX=/run/media/kf/WD/wine/vst/PluginAllianceBPE winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/PluginAllianceBPE wine s
yabridgectl add /run/media/kf/WD/wine/vst/PluginAllianceBPE/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/PluginAllianceFAS wine 
yabridgectl add /run/media/kf/WD/wine/vst/PluginAllianceFAS/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/PluginAllianceTTS winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/PluginAllianceTTS wine 
yabridgectl add /run/media/kf/WD/wine/vst/PluginAllianceTTS/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/PluginAllianceCopy winecfg
yabridgectl add /run/media/kf/WD/wine/vst/PluginAllianceCopy/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## PolyverseWider
WINEPREFIX=/run/media/kf/WD/wine/vst/PolyverseWider winetricks dxbk
WINEPREFIX=/run/media/kf/WD/wine/vst/PolyverseWider wine 
yabridgectl add /run/media/kf/WD/wine/vst/PolyverseWider/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## pulsar
no success
## RDGA
WINEPREFIX=/run/media/kf/WD/wine/vst/RDGA wine 
yabridgectl add /run/media/kf/WD/wine/vst/RDGA/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## slatedigital
WINEPREFIX=/run/media/kf/WD/wine/vst/Slatedigital winecfg
yabridgectl add /run/media/kf/WD/wine/vst/Slatedigital/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/Slatedigitalana2 winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/Slatedigitalana2 wine 
yabridgectl add /run/media/kf/WD/wine/vst/Slatedigitalana2/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## softube
no success

## ssl
WINEPREFIX=/run/media/kf/WD/wine/vst/SSL winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/SSL winecfg
yabridgectl add /run/media/kf/WD/wine/vst/SSL/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## soundtoys
no success

## sonible
WINEPREFIX=/run/media/kf/WD/wine/vst/SonibleDeEss winetricks mfc42 
WINEPREFIX=/run/media/kf/WD/wine/vst/SonibleDeEss wine 
yabridgectl add /run/media/kf/WD/wine/vst/SonibleDeEss/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/SonibleSmartEQ4 winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/SonibleSmartEQ4 wine 
yabridgectl add /run/media/kf/WD/wine/vst/SonibleSmartEQ4/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/Sonible winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/Sonible wine 
yabridgectl add /run/media/kf/WD/wine/vst/Sonible/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## Soundtoys
WINEPREFIX=/run/media/kf/WD/wine/vst/Soundtoys wine 
yabridgectl add /run/media/kf/WD/wine/vst/Soundtoys/drive_c/Program\ Files\ \(x86\)/VstPlugins/
yabridgectl sync --prune

## SpectrasonicsOmnishpere
WINEPREFIX=/run/media/kf/WD/wine/vst/SpectrasonicsOmnisphere winetricks  dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/SpectrasonicsOmnisphere wine 
yabridgectl add /run/media/kf/WD/wine/vst/SpectrasonicsOmnisphere/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## TDeEsser
WINEPREFIX=/run/media/kf/WD/wine/vst/TDeEsser wine 
yabridgectl add /run/media/kf/WD/wine/vst/TDeEsser/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## unitedplugin
WINEPREFIX=/run/media/kf/WD/wine/vst/UnitedPluginFS wine 
yabridgectl add /run/media/kf/WD/wine/vst/UnitedPluginFS/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/UnitedPluginIA wine
yabridgectl add /run/media/kf/WD/wine/vst/UnitedPluginIA/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/UnitedPluginJMGS wine
yabridgectl add /run/media/kf/WD/wine/vst/UnitedPluginJMGS/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/UnitedPluginMA wine
yabridgectl add /run/media/kf/WD/wine/vst/UnitedPluginMA/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/UnitedPluginSD wine
yabridgectl add /run/media/kf/WD/wine/vst/UnitedPluginSD/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## ValhallaDSP
WINEPREFIX=/run/media/kf/WD/wine/vst/ValhallaDSP wine 
yabridgectl add /run/media/kf/WD/wine/vst/ValhallaDSP/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## voxengo
WINEPREFIX=/run/media/kf/WD/wine/vst/Voxengo winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/Voxengo wine 
yabridgectl add /run/media/kf/WD/wine/vst/Voxengo/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## waves
no success

## WAKSHMRReverb
WINEPREFIX=/run/media/kf/WD/wine/vst/WAKSHMRReverb wine 
yabridgectl add /run/media/kf/WD/wine/vst/WAKSHMRReverb/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## wavesfactory
WINEPREFIX=/run/media/kf/WD/wine/vst/WavesFactory winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/WavesFactory wine 
yabridgectl add /run/media/kf/WD/wine/vst/WavesFactory/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## XLNRC20
WINEPREFIX=/run/media/kf/WD/wine/vst/XLNRC20 winetricks mfc42 
WINEPREFIX=/run/media/kf/WD/wine/vst/XLNRC20 wine 
yabridgectl add /run/media/kf/WD/wine/vst/XLNRC20/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## youlean 
WINEPREFIX=/run/media/kf/WD/wine/vst/Youlean wine 
yabridgectl add /run/media/kf/WD/wine/vst/Youlean/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## zynaptiaa
No success
WINEPREFIX=/run/media/kf/WD/wine/vst/Zynaptia winetricks mfc42 vcrun2022
WINEPREFIX=/run/media/kf/WD/wine/vst/Zynaptia wine 
yabridgectl add /run/media/kf/WD/wine/vst/Zynaptia/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

## 添加 vst3 插件
yabridgectl add path/to/vst3/plugin
yabridgec tl sync --prune

```

# wine


In [ ]:
!sudo pacman -Sy --needed --noconfirm wine winetricks wine-mono wine_gecko

In [ ]:
%%bash
wine_dir=$(pwd)/user/.wine
mkdir -p $wine_dir
echo $wine_dir

In [ ]:
%%bash
wine_dir=$(pwd)/user/.wine
rm $wine_dir/XStudio -r

In [ ]:
%%bash
## xstudio
wine_dir=$(pwd)/user/.wine
WINEPREFIX=$wine_dir/XStudio winetricks vcrun2019 dotnet48 corefonts dxvk cjkfonts

Executing cd /usr/bin


In [ ]:

## GTK+工具
sudo pacman -Sy --needed --noconfirm zenity  
sudo pacman -Sy --needed --noconfirm gnutls lib32-gnutls lib32-alsa-lib lib32-pulseaudio 

## xstudio
# 安装必要的依赖
wine_dir=$(pwd)/user/.wine 
WINEPREFIX=$wine_dir/XStudio winetricks vcrun2019 dotnet48 corefonts dxvk cjkfonts
WINEPREFIX=/run/media/kf/sdb/Linux/exe/XStudio wine  
## wechat
WINEPREFIX=/run/media/kf/sdb/Linux/exe/WeChat winetricks dxvk vcrun2022  cjkfonts
WINEPREFIX=/run/media/kf/sdb/Linux/exe/WeChat wine 


# Uninstall

In [ ]:
%%bash
sudo pacman -Rns code --noconfirm

# delete the folder
folder_path=$(pwd)/bin/vscode
rm -rf $folder_path
# delete the pth in ~/.zshrc
sed -i '/^export.*vscode/d' ~/.zshrc

